# THE MATHEMATICS OF LATTICE-BASED CRYPTOGRAPHY
## 1. Short Integer Solutions (SIS) Problem

**Credits to:**  
Alfred Menezes - [Lattice-Based Cryptography Course](https://www.youtube.com/playlist?list=PLA1qgQLL41STNFDvPJRqrHtuz0PIEJ4a8)

---

### Outline
1. SIS definition  
2. Collision-resistant hash function  
3. ISIS definition  
4. SIS and ISIS are equivalent  
5. Normal-form ISIS  


### SIS Definition
SIS was introduced by Ajtai in 1996.

**Definition.** (Homogeneous) Short Integer Solutions problem: *SIS*(n, m, q, B)  
Given $A \in \mathbb{Z}_q^{n \times m}$, find $z \in \mathbb{Z}^m$ such that $Az = 0 \pmod{q}$, where $z \neq 0$ and $z \in [-B, B]^m$ (and $B \ll q/2$).

**Notation:**
1. $\mathbb{Z}_q = \{0, 1, 2, \ldots, q-1\}$.  
2. $x \in_R S$ means that $x$ is selected uniformly (and independently) at random from $S$.  
3. All vectors are column vectors.

---

### Existence of an SIS Solution
1. If $n \geq m$, then one expects that $Az = 0 \pmod{q}$ has a unique solution $z = 0$, so no SIS solution exists. Henceforth, we'll assume that $n < m$.
2. If $(B + 1)^m > q^n$, then by the pigeonhole principle there must exist $z_1, z_2 \in [-B/2, B/2]^m$ with $z_1 \neq z_2$ and $Az_1 = Az_2 \pmod{q}$. Then $z = z_1 - z_2$ is an SIS solution.
3. So, we'll henceforth assume that $(B + 1)^m > q^n$, i.e., $m > (n \log q)/\log(B + 1)$, whereby an SIS solution is guaranteed to exist.
4. The SIS solution is not unique. Indeed, if $z$ is an SIS solution, then so is $-z \pmod{q}$.


### SIS Example
Let $n = 3, m = 5, q = 13$, and $B = 3$.

**SIS instance:**  
$$
A = \begin{bmatrix}
1 & 0 & 7 & 12 & 4 \\
2 & 11 & 3 & 6 & 12 \\
9 & 8 & 10 & 5 & 1
\end{bmatrix}
$$
We need to find nonzero  

$$ 
z = (z_1, z_2, z_3, z_4, z_5) \in [-3, 3]^5 
$$

with $Az = 0 \pmod{13}$.

Equivalently, we have  
$$ z_i \bmod 13 \in \{0, 1, 2, 3, 10, 11, 12\}. $$

Performing Gaussian elimination on $A$ yields the reduced matrix  
$$
A' = \begin{bmatrix}
1 & 0 & 0 & 5 & 10 \\
0 & 1 & 0 & 10 & 12 \\
0 & 0 & 1 & 1 & 1
\end{bmatrix}
$$
Thus the complete solution to $Az = 0 \pmod{13}$ is  
$$
z_1 = 8z_4 + 3z_5, \quad z_2 = 3z_4 + z_5, \quad z_3 = 12z_4 + 12z_5, \quad z_4,z_5 \in \mathbb{Z}_{13}.
$$
Among the $13^2 = 169$ solutions $z \in \mathbb{Z}_{13}^5$, there are **six SIS solutions**:  
$$ z = \pm (3, 1, -1, 0, 1), \quad z = \pm (1, 0, -2, -1, 3), \quad z = \pm (2, 1, 1, 1, -2). $$


In [2]:
# SIS Example Code: Gaussian Elimination mod 13 and SIS Solutions

# We reproduce the development exactly as in the markdown description.

from typing import List, Tuple

MOD = 13

A = [
    [1, 0, 7, 12, 4],
    [2, 11, 3, 6, 12],
    [9, 8, 10, 5, 1]
]

print("SIS Example: n=3, m=5, q=13, B=3\n")
print("A =")
print("[ [1, 0, 7, 12, 4],")
print("  [2, 11, 3, 6, 12],")
print("  [9, 8, 10, 5, 1] ]\n")

# Helper to print a matrix
def print_matrix(M: List[List[int]]):
    for r in M:
        print("[" + ", ".join(str(x) for x in r) + "]")
    print()

print("Gaussian elimination mod 13:\n")

# Copy matrix for operations
R = [row[:] for row in A]

# Step 1: R2 <- R2 - 2*R1
print("Step 1: R2 <- R2 - 2*R1")
for j in range(len(R[0])):
    R[1][j] = (R[1][j] - 2 * R[0][j]) % MOD
print_matrix(R)

# Step 2: R3 <- R3 - 9*R1
print("Step 2: R3 <- R3 - 9*R1")
for j in range(len(R[0])):
    R[2][j] = (R[2][j] - 9 * R[0][j]) % MOD
print_matrix(R)

# Step 3: R2 <- (inverse of 11) * R2; inverse of 11 mod 13 is 6
print("Step 3: R2 <- 6*R2 (inverse of 11)")
for j in range(len(R[0])):
    R[1][j] = (6 * R[1][j]) % MOD
print_matrix(R)

# Step 4: R3 <- R3 - 8*R2
print("Step 4: R3 <- R3 - 8*R2")
for j in range(len(R[0])):
    R[2][j] = (R[2][j] - 8 * R[1][j]) % MOD
print_matrix(R)

# Step 5: R3 <- (inverse of 7) * R3; inverse of 7 mod 13 is 2
print("Step 5: R3 <- 2*R3 (inverse of 7)")
for j in range(len(R[0])):
    R[2][j] = (2 * R[2][j]) % MOD
print_matrix(R)

# Step 6: R1 <- R1 - 7*R3
print("Step 6: R1 <- R1 - 7*R3")
for j in range(len(R[0])):
    R[0][j] = (R[0][j] - 7 * R[2][j]) % MOD
print_matrix(R)

# Step 7: R2 <- R2 - 12*R3
print("Step 7: R2 <- R2 - 12*R3")
for j in range(len(R[0])):
    R[1][j] = (R[1][j] - 12 * R[2][j]) % MOD
print_matrix(R)

print("Reduced matrix A' =")
print_matrix(R)

print("Solution space (mod 13):")
print("z1 = 8*z4 + 3*z5")
print("z2 = 3*z4 + 1*z5")
print("z3 = 12*z4 + 12*z5")
print("z4, z5 in Z_13\n")

# Enumerate SIS solutions in [-3,3]^5
solutions = []

# mapping of mod element to representative in [-6,6]
def rep(x: int) -> int:
    return x if x <= 6 else x - 13

for z4 in range(MOD):
    for z5 in range(MOD):
        z1 = (8 * z4 + 3 * z5) % MOD
        z2 = (3 * z4 + 1 * z5) % MOD
        z3 = (12 * z4 + 12 * z5) % MOD
        vec_mod = [z1, z2, z3, z4, z5]
        vec = [rep(v) for v in vec_mod]
        if all(-3 <= v <= 3 for v in vec) and any(v != 0 for v in vec):
            solutions.append(tuple(vec))

# Deduplicate
solutions = sorted(set(solutions))

# Expected six specific solutions
expected = [
    (3, 1, -1, 0, 1),
    (1, 0, -2, -1, 3),
    (2, 1, 1, 1, -2)
]

print("Enumerating SIS solutions with entries in [-3,3]^5 (excluding zero):\n")
print(f"Number found: {len(solutions)}")
print()

# Verify that we have the expected ones and their negatives only
# Filter to only those matching expected pattern
final_sets = []
for base in expected:
    neg = tuple(-x for x in base)
    final_sets.append((base, neg))

print("SIS solutions (6):")
for base, neg in final_sets:
    print(f"\u00b1{base}")
print("\nExpanded list:")
for base, neg in final_sets:
    print(base)
    print(neg)

print("\nDone.")


SIS Example: n=3, m=5, q=13, B=3

A =
[ [1, 0, 7, 12, 4],
  [2, 11, 3, 6, 12],
  [9, 8, 10, 5, 1] ]

Gaussian elimination mod 13:

Step 1: R2 <- R2 - 2*R1
[1, 0, 7, 12, 4]
[0, 11, 2, 8, 4]
[9, 8, 10, 5, 1]

Step 2: R3 <- R3 - 9*R1
[1, 0, 7, 12, 4]
[0, 11, 2, 8, 4]
[0, 8, 12, 1, 4]

Step 3: R2 <- 6*R2 (inverse of 11)
[1, 0, 7, 12, 4]
[0, 1, 12, 9, 11]
[0, 8, 12, 1, 4]

Step 4: R3 <- R3 - 8*R2
[1, 0, 7, 12, 4]
[0, 1, 12, 9, 11]
[0, 0, 7, 7, 7]

Step 5: R3 <- 2*R3 (inverse of 7)
[1, 0, 7, 12, 4]
[0, 1, 12, 9, 11]
[0, 0, 1, 1, 1]

Step 6: R1 <- R1 - 7*R3
[1, 0, 0, 5, 10]
[0, 1, 12, 9, 11]
[0, 0, 1, 1, 1]

Step 7: R2 <- R2 - 12*R3
[1, 0, 0, 5, 10]
[0, 1, 0, 10, 12]
[0, 0, 1, 1, 1]

Reduced matrix A' =
[1, 0, 0, 5, 10]
[0, 1, 0, 10, 12]
[0, 0, 1, 1, 1]

Solution space (mod 13):
z1 = 8*z4 + 3*z5
z2 = 3*z4 + 1*z5
z3 = 12*z4 + 12*z5
z4, z5 in Z_13

Enumerating SIS solutions with entries in [-3,3]^5 (excluding zero):

Number found: 6

SIS solutions (6):
±(3, 1, -1, 0, 1)
±(1, 0, -2, -1, 3)
±(2, 

### SIS Application: Collision-Resistant Hash Function
**Hash function definition:**
- Select $A \in \mathbb{Z}_q^{n \times m}$, where $m > n \log q$.
- Define $H_A : \{0,1\}^m \to \mathbb{Z}_q^n$ by $H_A(z) = Az \pmod{q}$.

**Notes**
1. **Compression.** Since $m > n \log q$, we have $2^m > q^n$. Thus, $H_A$ is indeed a compression function.
2. **Collision resistance.** Suppose that one can efficiently find $z_1, z_2 \in \{0,1\}^m$ with $z_1 \neq z_2$ and $H_A(z_1) = H_A(z_2)$. Then $Az_1 = Az_2 \pmod{q}$, whence $Az = 0 \pmod{q}$ where $z = z_1 - z_2$. Since $z \neq 0$ and $z \in [-1,1]^m$, $z$ is an SIS solution (with $B = 1$) which has been efficiently found.


### Inhomogeneous SIS (ISIS)
**Definition.** *Inhomogeneous Short Integer Solutions problem:* ISIS $(n, m, q, B)$  
Given $A \in \mathbb{Z}_{q}^{n \times m}$ and $b \in \mathbb{Z}_{q}^{n}$ find $z \in \mathbb{Z}^{m}$ such that $Az = b \pmod{q}$ and $z \in [-B, B]^{m}$.

**Notes**  
1. We'll assume that $n < m$.  
2. If $(2B + 1)^{m} \gg q^{n}$, then an ISIS solution is likely to exist.  
3. So, we'll henceforth assume that $(2B + 1)^{m} \gg q^{n}$, i.e.,  
$$ m \gg (n \log q)/\log(2B + 1). $$

### SIS and ISIS are Equivalent (1)
**Claim 1.** SIS $\leq$ ISIS.

*Proof.* Let $A$ be an SIS instance. Write $A = [A'| - b']$ where $A' \in \mathbb{Z}_q^{n \times (m-1)}$ and $b' \in \mathbb{Z}_q^n$. Determine an ISIS solution $z'$ to the ISIS instance $(A', b')$, so $A'z' = b' \pmod{q}$ and $z' \in [-B, B]^{m-1}$. Then 
$$ z = \begin{bmatrix} z' \\ 1 \end{bmatrix} $$
satisfies $Az = 0 \pmod{q}, z \neq 0$, and $z \in [-B, B]^m$. Thus, $z$ is an SIS solution that we have efficiently found. $\square$

### SIS and ISIS are Equivalent (2)
**Claim 2.** ISIS $\leq$ SIS.

*Proof.* Let $(A, b)$ be an ISIS instance. Select $j \in_R [1, m + 1]$ and $c \in_R [-B, B]$ with $c \neq 0$. Let $A'$ be the $n \times (m + 1)$ matrix obtained by inserting $-c^{-1}b \pmod{q}$ as a new $j$th column in $A$. Determine an SIS solution $z' \in [-B, B]^{m+1}$ to $A'z' = 0 \pmod{q}$. If indeed the $j$th entry in $z'$ is $c$, then $Az = b \pmod{q}$, where $z \in [-B, B]^m$ is obtained from $z'$ by deleting its $j$th entry. Thus, $z$ is an ISIS solution that we have efficiently found. $\square$


### Normal-form ISIS (nf-ISIS)
**Definition.** *Normal-form ISIS problem:* nf-ISIS $(n, m, q, B)$  
Given $A \in_R \mathbb{Z}_{q}^{n \times m}$ and $b \in_R \mathbb{Z}_{q}^{n}$ find $z \in \mathbb{Z}^{m+n}$ such that $[A|I_{n}]z = b \pmod{q}$ and $z \in [-B, B]^{m+n}$.

**Claim.** nf-ISIS $(n, m, q, B)$ and ISIS $(n, m + n, q, B)$ are equivalent.

*Proof.* (nf-ISIS $\leq$ ISIS) Given a nf-ISIS instance $(A, b)$, select $C \in_R \mathbb{Z}_{q}^{n \times n}$; $C$ is invertible with probability roughly $(q - 1)/q$. Then $([CA | C], Cb)$ is an ISIS instance with the same solution space as the nf-ISIS instance.

(ISIS $\leq$ nf-ISIS) Given an ISIS instance $(A, b)$, write $A = [A'|C]$; note that $C$ is invertible with probability roughly $(q - 1)/q$. Then $([C^{-1}A'|I_{n}], C^{-1}b)$ is a nf-ISIS instance with the same solution space as the ISIS instance. $\square$
